In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# -----------------------------------------------------------------------------
# HELPER: Simulation Logic
# -----------------------------------------------------------------------------
def get_simulated_loss(epochs, noise_level=0.1):
    """Generates a synthetic loss curve for demonstration."""
    steps = np.linspace(0, epochs, 30)
    # Exponential decay to simulate learning
    loss = 2.5 * np.exp(-0.5 * steps) 
    # Add random noise
    loss += np.random.normal(0, noise_level, len(steps))
    return np.clip(loss, 0.1, 5.0)

# -----------------------------------------------------------------------------
# APP CLASS: Minimal UI
# -----------------------------------------------------------------------------
class SimpleFineTuner:
    def __init__(self):
        self.output_area = widgets.Output()
        
        # --- Controls ---
        self.epoch_slider = widgets.IntSlider(
            value=3, min=1, max=10, 
            description='Epochs:',
            continuous_update=False
        )
        
        self.batch_select = widgets.Dropdown(
            options=[4, 8, 16, 32], 
            value=8, 
            description='Batch Size:'
        )
        
        # --- Buttons ---
        self.btn_train = widgets.Button(
            description="Run Simulation", 
            button_style='success', # Green
            icon='play'
        )
        self.btn_code = widgets.Button(
            description="Show Requirement Code", 
            button_style='info',    # Blue
            icon='code'
        )
        
        # --- Event Binding ---
        self.btn_train.on_click(self.run_simulation)
        self.btn_code.on_click(self.show_code)
        
    def show_code(self, b):
        """Displays the raw code required for the assignment."""
        with self.output_area:
            clear_output()
            print("=== 📝 REQUIREMENT: FINE-TUNING CODE PATTERN ===")
            print("This code demonstrates how to use Hugging Face for the task:\n")
            
            code_str = """
# 1. Dataset Preparation
from datasets import load_dataset
dataset = load_dataset("imdb")

# 2. Model & Tokenizer Setup
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token # GPT-2 needs a pad token
model = AutoModelForCausalLM.from_pretrained("gpt2")

# 3. Preprocessing
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 4. Training Configuration
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8
)

# 5. Initialize & Run Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"]
)

trainer.train()
            """
            print(code_str)

    def run_simulation(self, b):
        """Runs the mock training loop."""
        epochs = self.epoch_slider.value
        
        with self.output_area:
            clear_output()
            print(f"🚀 Starting Mock Training ({epochs} Epochs)...")
            
            # Generate fake data
            loss_curve = get_simulated_loss(epochs)
            progress = widgets.FloatProgress(min=0, max=len(loss_curve), description="Training:")
            display(progress)
            
            # Simulation Loop
            for i, loss in enumerate(loss_curve):
                progress.value = i + 1
                if i % 5 == 0:
                    print(f"Step {i*10}: Loss = {loss:.4f}")
                time.sleep(0.1) # Simulate computation time
            
            # Final Result
            print("\n✅ Training Complete.")
            
            # Plot
            plt.figure(figsize=(8, 4))
            plt.plot(loss_curve, marker='o', linestyle='-', color='teal', label='Training Loss')
            plt.title("Simulated Loss Curve")
            plt.xlabel("Steps")
            plt.ylabel("Loss")
            plt.grid(True, alpha=0.3)
            plt.legend()
            plt.show()

    def render(self):
        """Displays the UI elements."""
        display(widgets.VBox([
            widgets.HTML("<h3>📉 Simple GPT Fine-Tuning Lab</h3>"),
            widgets.HBox([self.epoch_slider, self.batch_select]),
            widgets.HBox([self.btn_train, self.btn_code]),
            widgets.HTML("<hr>"),
            self.output_area
        ]))

# --- Run Application ---
if __name__ == "__main__":
    app = SimpleFineTuner()
    app.render()